# Generate Test Data 

Generate Test Data so that we Can Test our Preprocessing and Trainer Functions 

In [1]:
import os
import pprint
import tempfile
import urllib

import pandas as pd
from sklearn.model_selection import train_test_split

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components.example_gen.custom_executors import parquet_executor 
from tfx.components.example_gen.component import FileBasedExampleGen

from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2

from tensorflow_metadata.proto.v0 import schema_pb2 
from tensorflow_metadata.proto.v0 import statistics_pb2
from tensorflow_metadata.proto.v0 import anomalies_pb2

from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

import tensorflow_data_validation as tfdv 
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

/home/jupyter/venv/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [31]:
# Read in all the data
_data_root_split = "gs://ml-sandbox-tagging-tfx-experiments/data/split/"
_train_path = os.path.join(_data_root_split, "train.parquet")
_eval_path = os.path.join(_data_root_split, "eval.parquet")

train_data = pd.read_parquet(_train_path)
eval_data = pd.read_parquet(_eval_path)


# Take a sample of the data and write it out
train_sample = train_data.iloc[:10]
eval_sample = eval_data.iloc[:10]

_train_sample_path = "gs://ml-sandbox-tagging-tfx-experiments/testdata/parquet/train.parquet"
_eval_sample_path = "gs://ml-sandbox-tagging-tfx-experiments/testdata/parquet/eval.parquet"

train_sample.to_parquet(_train_sample_path, index=False)
eval_sample.to_parquet(_eval_sample_path, index=False)

In [34]:
train_sample.to_parquet('../src/testdata/parquet/train.parquet', index=False)
eval_sample.to_parquet('../src/testdata/parquet/eval.parquet', index=False)

In [35]:
_pipeline_root = tempfile.mkdtemp(prefix='pipeline')
_pipeline_name = 'make_test_data'

context = InteractiveContext(pipeline_name=_pipeline_name, pipeline_root=_pipeline_root)

In [3]:
input_config = example_gen_pb2.Input(splits=[
                    example_gen_pb2.Input.Split(name='train', pattern='train*'),
                    example_gen_pb2.Input.Split(name='eval', pattern='eval*')
])

example_gen = FileBasedExampleGen(input_base='gs://ml-sandbox-tagging-tfx-experiments/testdata/parquet', 
                                  custom_executor_spec=executor_spec.ExecutorClassSpec(parquet_executor.Executor),
                                  input_config=input_config)

context.run(example_gen)

ExecutionResult(
    component_id: FileBasedExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/pipelineno00rwel/FileBasedExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:7227,xor_checksum:1603788753,sum_checksum:1603788753\nsplit:eval,num_files:1,total_bytes:8203,xor_checksum:1603788753,sum_checksum:1603788753"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [4]:
train_path = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')
eval_path = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'eval')

In [5]:
!gsutil -m cp -r {train_path} gs://ml-sandbox-tagging-tfx-experiments/testdata/example_gen/train
!gsutil -m cp -r {eval_path} gs://ml-sandbox-tagging-tfx-experiments/testdata/example_gen/eval

Copying file:///tmp/pipelineno00rwel/FileBasedExampleGen/examples/1/train/data_tfrecord-00000-of-00001.gz [Content-Type=application/octet-stream]...
/ [1/1 files][  2.9 KiB/  2.9 KiB] 100% Done                                    
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///tmp/pipelineno00rwel/FileBasedExampleGen/examples/1/eval/data_tfrecord-00000-of-00001.gz [Content-Type=application/octet-stream]...
/ [1/1 files][  3.1 KiB/  3.1 KiB] 100% Done                                    
Operation completed over 1 objects/3.1 KiB.                                      


In [6]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/pipelineno00rwel/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [7]:
infer_schema = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                         infer_feature_shape=True)
context.run(infer_schema)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/pipelineno00rwel/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [8]:
schema_file = os.path.join(infer_schema.outputs['schema'].get()[0].uri, 'schema.pbtxt')

In [9]:
!gsutil cp {schema_file} gs://ml-sandbox-tagging-tfx-experiments/testdata/schema_gen/schema.pbtxt

Copying file:///tmp/pipelineno00rwel/SchemaGen/schema/3/schema.pbtxt [Content-Type=application/octet-stream]...
/ [1 files][ 11.5 KiB/ 11.5 KiB]                                                
Operation completed over 1 objects/11.5 KiB.                                     


---
## Transform

In [10]:
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema

In [11]:
_transform_module_file = 'transform.py'

In [12]:
%%writefile {_transform_module_file}

"""
This file contains the preprocessing_fn callback function for the transform 
component in the tfx pipeline
"""
import tensorflow_transform as tft
import tensorflow as tf

_FEATURE_KEY = 'features'
_LABEL_KEY = 'series_ep_tags'
MAX_STRING_LENGTH = 277

# Filter out everything other than German characters 
regex_filter = '[^äÄöÖüÜßa-zA-Z]'

def _transformed_name(key):
    return key + '_xf'

def clean(text):
    """
    Clean up text input by removing everything but German words 

    Args: 
        text: tensors containing the synopsis. (batch_size/None, 1)

    Returns: 
        final: Cleaned text 
    """
    # Encoding needed to keep german characters
    lower = tf.strings.lower(text, 'utf-8')
    cleaned = tf.strings.regex_replace(lower, regex_filter, ' ')
    # Filter single letters
    single_letters_removed = tf.strings.regex_replace(cleaned, r'((^|\s)[äÄöÖüÜßa-zA-Z]{1})+\s', ' ')
    stripped = tf.strings.strip(single_letters_removed)
    # Replace multiple spaces with single space
    final = tf.strings.regex_replace(stripped, ' +', ' ')

    return final

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs 
    
    Args: 
        inputs: map from feature keys to raw not-yet-transformed-features 

    Returns: 
        Map from string feature key to transformed feature operations 
    """
    outputs = {}
    
    text = tf.squeeze(inputs[_FEATURE_KEY], axis=1)
    tags = inputs[_LABEL_KEY]

    cleaned = clean(text)
    text_tokens = tf.compat.v1.string_split(cleaned, ' ', result_type='RaggedTensor')
    text_tokens = text_tokens.to_tensor(shape=[None, MAX_STRING_LENGTH])
    text_indices = tft.compute_and_apply_vocabulary(
        text_tokens, vocab_filename='vocab', num_oov_buckets=1
    )
    
    # compute vocab of tags, do not apply due to serving issues
    _ = tft.vocabulary(tags, vocab_filename='tags')
    
    outputs[_transformed_name(_FEATURE_KEY)] = text_indices
    outputs[_LABEL_KEY] = tags
    
    return outputs

Overwriting transform.py


In [13]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    module_file='./transform.py'
)
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/pipelineno00rwel/Transform/transform_graph/4/.temp_path/tftransform_tmp/ecb691387a764981bd6d883533a6b556/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/pipelineno00rwel/Transform/transform_graph/4/.temp_path/tftransform_tmp/b5b86839a6c34427bdf77c57c76c0c07/assets
INFO:tensorflow:SavedModel written to: /tmp/pipelineno00rwel/Transform/transform_graph/4/.temp_path/tftransform_tmp/b5b86839a6c34427bdf77c57c76c0c07/saved_model.pb
value: "\n\013\n\tConst_1:0\022\005vocab"

value: "\n\013\n\tConst_3:0\022\004tags"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
value: "\n\013\n\tConst_1:0\022\005vocab"

value: "\n\013\n\tConst_3:0\022\004tags"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
value: "\n\013\n\tConst_1:0\022\005vocab"

value: "\n\013\n\tConst_3:0\022\004tags"

INFO:tensorflow:Saver not created 

ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/pipelineno00rwel/Transform/transform_graph/4"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 5
        type_id: 5
        uri: "/tmp/pipelineno00rwel/Transform/transformed_examples/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 6
        type_id: 12
        uri: "/tmp/pipelineno00rwel/Transform/updated_analyzer_cache/4"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 12
        name: "TransformCache"
        )]
        ))

In [25]:
transform_output = transform.outputs['transform_graph'].get()[0].uri
transform_examples = transform.outputs['transformed_examples'].get()[0].uri

In [24]:
!gsutil -m cp -r {transform_output} gs://ml-sandbox-tagging-tfx-experiments/testdata/transform_output

Copying file:///tmp/pipelineno00rwel/Transform/transform_graph/4/transform_fn/saved_model.pb [Content-Type=application/octet-stream]...
Copying file:///tmp/pipelineno00rwel/Transform/transform_graph/4/transform_fn/assets/tags [Content-Type=application/octet-stream]...
Copying file:///tmp/pipelineno00rwel/Transform/transform_graph/4/transform_fn/assets/vocab [Content-Type=application/octet-stream]...
Copying file:///tmp/pipelineno00rwel/Transform/transform_graph/4/transformed_metadata/schema.pbtxt [Content-Type=application/octet-stream]...
Copying file:///tmp/pipelineno00rwel/Transform/transform_graph/4/metadata/schema.pbtxt [Content-Type=application/octet-stream]...
/ [5/5 files][ 34.6 KiB/ 34.6 KiB] 100% Done                                    
Operation completed over 5 objects/34.6 KiB.                                     


In [26]:
!gsutil -m cp -r {transform_examples} gs://ml-sandbox-tagging-tfx-experiments/testdata/transformed_examples

Copying file:///tmp/pipelineno00rwel/Transform/transformed_examples/4/train/transformed_examples-00000-of-00001.gz [Content-Type=application/octet-stream]...
Copying file:///tmp/pipelineno00rwel/Transform/transformed_examples/4/eval/transformed_examples-00000-of-00001.gz [Content-Type=application/octet-stream]...
/ [2/2 files][  3.5 KiB/  3.5 KiB] 100% Done                                    
Operation completed over 2 objects/3.5 KiB.                                      


In [37]:
!gsutil -m cp -r gs://ml-sandbox-tagging-tfx-experiments/testdata ../src

Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/example_gen/eval/data_tfrecord-00000-of-00001.gz...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/example_gen/train/data_tfrecord-00000-of-00001.gz...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/example_gen/eval/eval/data_tfrecord-00000-of-00001.gz...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/example_gen/train/train/data_tfrecord-00000-of-00001.gz...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/parquet/eval.parquet...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/parquet/train.parquet...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/schema_gen/schema.pbtxt...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/transform_output/transform_fn/saved_model.pb...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/transformed_examples/eval/transformed_examples-00000-of-00001.gz...
Copying gs://ml-sandbox-tagging-tfx-experiments/testdata/transform_output/

In [ ]:
!rm transform.py